## **Dhaka Traffic Detection using YOLOv5**

## ***Clone YOLOv5 repository***

In [ ]:
%cd /content/
!git clone https://github.com/Morshed-Alam/yolov5.git  # clone repo
!pip install -qr yolov5/requirements.txt  # install dependencies (ignore errors)
%cd yolov5
 
import torch
from IPython.display import Image, clear_output  # to display images
from utils.google_utils import gdrive_download  # to download models/datasets
 
!rm -r /content/sample_data/
clear_output()
print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

## ***Download yolo formatted traffic data from google drive***

In [ ]:
%cd /content/
!gdown --id 1RNL2AT0UIrmQl7j0Ul01wVO1tKbBO1lG
!unzip dataset1.zip; rm dataset1.zip;

## ***Define model configuration and architecture***

***Read number of classes from data.yaml***

In [ ]:
import yaml
with open("/content/data.yaml", 'r') as stream:
  num_classes = str(yaml.safe_load(stream)['nc'])
  print(num_classes)

In [ ]:
#Customize IPython write file so we can write variables
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
  with open(line, 'w') as f:
    f.write(cell.format(**globals()))

***Modify number of classes***

> Other parameters are same as yolov5x.yaml



In [ ]:
%%writetemplate /content/yolov5/models/custom_yolov5x.yaml

# parameters
nc: {num_classes} # Number of classes
depth_multiple: 1.33 # model depth multiple
width_multiple: 1.25 # layer channel multiple

# Anchors
anchors:
 - [10,13, 16,30, 33,23]  # P3/8
 - [30,61, 62,45, 59,119]  # P4/16
 - [116,90, 156,198, 373,326]  # P5/32

# YOLOv5 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Focus, [64, 3]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, BottleneckCSP, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 9, BottleneckCSP, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, BottleneckCSP, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 1, SPP, [1024, [5, 9, 13]]],
   [-1, 3, BottleneckCSP, [1024, False]],  # 9
  ]

# YOLOv5 head
head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, BottleneckCSP, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]],  # cat backbone P3
   [-1, 3, BottleneckCSP, [256, False]],  # 17 (P3/8-small)

   [-1, 1, Conv, [256, 3, 2]],
   [[-1, 14], 1, Concat, [1]],  # cat head P4
   [-1, 3, BottleneckCSP, [512, False]],  # 20 (P4/16-medium)

   [-1, 1, Conv, [512, 3, 2]],
   [[-1, 10], 1, Concat, [1]],  # cat head P5
   [-1, 3, BottleneckCSP, [1024, False]],  # 23 (P5/32-large)

   [[17, 20, 23], 1, Detect, [nc, anchors]],  # Detect(P3, P4, P5)
  ]

***Specify Hyperparemeters***

> Augmentation hyperparameters tuned on PASCAL VOC dataset are used here



In [ ]:
%%writetemplate /content/yolov5/data/custom_hyp.yaml
lr0: 0.01
lrf: 0.2
momentum: 0.937
weight_decay: 0.0005
warmup_epochs: 3.0
warmup_momentum: 0.8
warmup_bias_lr: 0.1
box: 0.05
cls: 0.5
cls_pw: 1.0
obj: 1.0
obj_pw: 1.0
iou_t: 0.20
anchor_t: 4.0
#anchors: 3.63
fl_gamma: 0.0
hsv_h: 0.0138
hsv_s: 0.664
hsv_v: 0.464
degrees: 0.373
translate: 0.245
scale: 0.898
shear: 0.602
perspective: 0.0
flipud: 0.00856
fliplr: 0.5
mosaic: 1.0
mixup: 0.243

## ***Train custom YOLOv5 detector***

***Log in to wandb***


In [ ]:
!pip install wandb -qqq
import wandb
wandb.login

In [ ]:
# train yolov5s on custom data for 40 epochs
# time its performance
%%time
%cd /content/yolov5/
!python train.py --img-size 1024 \
                 --batch-size 4 \
                 --epochs 40 \
                 --data ../data.yaml \
                 --cfg ./models/custom_yolov5x.yaml \
                 --weights ./weights/yolov5x.pt \